In [16]:
# Install libraries
%%capture
!pip install tensorflow-probability
!pip install imageio

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
%cd gdrive/My Drive/COMP3710/PatternFlow/recognition/VQVAE_oasis3

[Errno 2] No such file or directory: 'gdrive/My Drive/COMP3710/PatternFlow/recognition/VQVAE_oasis3'
/content/gdrive/My Drive/COMP3710/PatternFlow/recognition


In [22]:
! git config --global user.email "rweldridge99@gmail.com"
! git commit -m "Committed a starting file with progress on loading the OASIS dataset into tf.data.Dataset objects."

[master f5ae4ac] Committed a starting file with progress on loading the OASIS dataset into tf.data.Dataset objects.
 1 file changed, 1 insertion(+)
 create mode 100644 recognition/VQVAE_oasis3/VQVAE.ipynb


In [24]:
! git push

fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
# Import General Utilities
import numpy as np
import os
import random
from tqdm.notebook import tqdm, trange
import time
import datetime

#Import tensorflow and its requirements
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_datasets as tfds

# Import plotting tools and image converters
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import PIL
import imageio
from IPython import display

# Import file path variable handling
from pathlib import Path
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

# Constants
# TO ADD %%%%%%%%%%%%%%%%%%%%%%%%%%

print(tf.__version__)

2.6.0


In [ ]:
# Download OASIS Dataset from the provided link

def download_and_unzip(url, extract_to='.'): # Credit to Antoine Hebert
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)
    http_response.close()

root_dir = '/root/.keras/datasets'
_URL = "https://cloudstor.aarnet.edu.au/plus/s/tByzSZzvvVh0hZA/download"
download_and_unzip(_URL, extract_to=root_dir)
#data_dir = tf.keras.utils.get_file('oasis3', origin=_URL, cache_subdir=root_dir, extract=True)
data_dir = Path(root_dir) / 'keras_png_slices_data'

print(f"Data Directory is {data_dir}")
print(f"Contents: \n\r {os.listdir(data_dir)}")

# Folder Directory Paths
train_dir = data_dir / 'keras_png_slices_train'
train_ans_dir = data_dir / 'keras_png_slices_seg_train'
test_dir = data_dir / 'keras_png_slices_test'
test_ans_dir = data_dir / 'keras_png_slices_seg_test'
val_dir = data_dir / 'keras_png_slices_validate'
val_ans_dir = data_dir / 'keras_png_slices_seg_validate'

Data Directory is /root/.keras/datasets/keras_png_slices_data
Contents: 
 ['keras_png_slices_train', 'keras_png_slices_seg_train', 'keras_png_slices_seg_test', 'keras_png_slices_validate', 'keras_png_slices_seg_validate', 'keras_png_slices_test']


In [ ]:
def count_files(dat_path):
    return len(list(dat_path.glob('*.*')))

train_ds_list = tf.data.Dataset.list_files(str(train_dir/'*.*'), shuffle=False)
train_ds_list = train_ds_list.shuffle(count_files(train_dir), reshuffle_each_iteration=False)

for f in train_ds_list.take(5):
  print(f.numpy())

b'/root/.keras/datasets/keras_png_slices_data/keras_png_slices_train/case_353_slice_19.nii.png'
b'/root/.keras/datasets/keras_png_slices_data/keras_png_slices_train/case_318_slice_30.nii.png'
b'/root/.keras/datasets/keras_png_slices_data/keras_png_slices_train/case_338_slice_9.nii.png'
b'/root/.keras/datasets/keras_png_slices_data/keras_png_slices_train/case_256_slice_6.nii.png'
b'/root/.keras/datasets/keras_png_slices_data/keras_png_slices_train/case_289_slice_3.nii.png'
